# **Install next libraries**

In [1]:
%pip install --quiet langchain-google-genai pillow langchain-community pypdf faiss-cpu langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **Importings**

In [2]:
import os
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from IPython.display import Markdown as md
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide Gemini API Key: ")

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [5]:
class PDFReader:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def GetPages(self):
        loader = PyPDFLoader(self.pdf_path)
        return loader.load_and_split()
    
    def CreateFAISSIndex(self):
        vector_db = FAISS.from_documents(self.GetPages(), GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
        return vector_db

In [10]:
pdf_file = PDFReader("pdf/example1.pdf")
db = pdf_file.CreateFAISSIndex()
#docs = db.similarity_search("Que es el liquido sinovial?", k=10)
#for doc in docs:
#    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

In [11]:
TEMPLATE = """ 
Answer the question based only on the following context:
{context}

Question: {question}
"""

In [19]:
prompt_template = ChatPromptTemplate.from_template(TEMPLATE)
retriever = db.as_retriever()
parser = StrOutputParser()
setup_and_retrieval = RunnableParallel({
    "context" : retriever, "question" : RunnablePassthrough()
})
chain = setup_and_retrieval | prompt_template | llm | parser
md(chain.invoke("Cuales son las funciones del liquido sinovial?"))

El líquido sinovial tiene las siguientes funciones:

* **Nutrición del cartílago articular:** El líquido sinovial se encarga de llevar nutrientes al cartílago, que no tiene vasos sanguíneos propios.
* **Eliminación de desechos:** El líquido sinovial también elimina los desechos que se producen en el cartílago.
* **Función protectora:** El líquido sinovial es viscoso, lo que ayuda a reducir la fricción entre los cartílagos durante el movimiento. También sirve como amortiguador para proteger la articulación de impactos. 
